In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
from tensorflow import keras
from tensorflow.keras import callbacks, models

In [2]:
#回呼(callbacks): 呼叫fit()方法時傳遞給模型的一個物件(一個執行特定方法的物件)，在訓練期間由模型在各節點呼叫
#可在發現測量驗證損失不再改善時即停止訓練、儲存模型、載入不同的權重、更改模型的狀態
#可使用回乎的範例:
#1.模型檢查點(model checkpoint): 訓練時間保存模型在不同時間點的權重
#2.早期停止(early stopping): 當驗證損失不再改善時中斷訓練，並自動保存訓練時間獲得的最佳模型
#3.在訓練期間動態調整某些參數的值，ex:優化器的學習效率
#4.在訓練期間記錄訓練和驗證指標，或在模型更新時視覺化模型已學習到的表示法(權重)，ex:Keras透過回呼處理進度條圖

In [3]:
#keras.callbacls.LearningRateScheduler: 自訂函數，動態調整學習率，ex:10個epoch前，學習率不變；10之後的每一個epoch學習率變之前的一半
#keras.callbacks.CSVLogger: 保存訓練的loss和accuracy等資訊到csv檔
#keras.callbacks.TerminateOnNaN：loss變成NaN時停止，通常發生在學習率過大開始發散時

In [4]:
#早期停止EarlyStopping 和 模型檢查點ModelCheckpoint 回呼
#當監控的指標(ex:驗證損失)在一定數量的訓練週期(次數)停止改善，即可使用EarlyStopping回呼來中斷訓練
#結合ModelCheckPoint回呼使用，在訓練期間不斷儲存模型(可選擇保存最佳的模型版本)

In [6]:
#用串列打包回呼，傳遞給模型
                #回呼1
callbacks_list = [callbacks.EarlyStopping(monitor='val_acc', #監控模型的驗證精準度
                                               patience=1), #當監控的指標超過1個epochs未改善則停止訓練
                #回呼2
                 callbacks.ModelCheckpoint(filepath='my_model.h5', #每個epochs結束後儲存模型
                                          monitor='val_loss', #當val_loss取得改善時，回呼才會儲存模型
                                          save_best_only=True)]

In [ ]:
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['acc'])

In [ ]:
model.fit(x, y,
         validation_data=(x_val, y_val),
         epochs=10,
         batch_size=32,
         callbacks=callbacks_list)

In [ ]:
#ReduceLROnPlateau回呼:當驗證損失停止改善時(進入高原期plateau)，可使用此回呼降低學習率
#在損失停滯(loss plateau)的情況下，降低或提高學習率式訓練期間擺脫局部最小值的有效策略

In [ ]:
callbacks_list = [callbacks.ReduceLROnPlateau(monitor='val_loss', #監控模型的驗證損失
                                             factor=0.1, #模型呼叫回呼時，將學習率*0.1
                                             patience=10)] #當監控的指標超過10個epochs未改善則呼叫回呼

In [ ]:
model.fit(x, y,
         validation=(x_val, y_val),
         epochs=10,
         batch_size=32,
         callbacks=callbacks_list)

In [ ]:
#撰寫自己的回呼: 透過繼承keras.callbacks.Callback類別，編寫自己需要的method
#這些method在呼叫時都會使用logs參數，這是一個字典(dict)，包含上一批次量、訓練週期(次數)、訓練與驗證指標...
#回呼會存取以下屬性:
#self.model: 呼叫回呼的模型物件
#self.validation_data: 傳遞給fit作為驗證資料的數值

#method需要明確的命名:
#on_epoch_begin:在每個訓練週期(次數)的開始時呼叫
#on_epoch_end:在每個訓練週期(次數)的結束時呼叫
#on_batch_begin:在處理每個批次量前呼叫
#on_batch_end:在處理每個批次量後呼叫
#on_train_begin:在訓練開始時呼叫
#on_train_end:在訓練結束時呼叫

In [ ]:
#此回呼在每個訓練週期(次數)結束時，以驗證資料集的第一個樣本，計算模型的每一層啟動函數結果，並儲存到磁碟中(以Numpy陣列形式)
class ActivationLogger(callbacks.Callback): #繼承Callback類別
    def set_model(self, model):
        self.model = model #訓練前由父模型呼叫，以設定哪個模型要使用回呼
        layers_outputs = [layer.output for layer in model.layers] #取得模型中每一層輸出
        self.activations_model = models.Model(model.input, layer_outputs) #透過輸入與每一層的輸出來建立每一層的Model物件
        
    def on_epoch_end(self, epoch, logs=None): #在每個訓練週期結束時呼叫
        if self.validation is None:
            raise RuntimeError('Requires validation_data')
        
        validation_sample = self.validation_data[0][0:1] #取得驗證集的第一個樣本
        activations = self.activation_model.predict(validation_sample) #將此樣本送入每一層Model物件，取得輸出結果(張量)
        #將結果張量儲存到磁碟中(.npz檔)
        f = open('activation_at_epoch_', str(epoch) + '.npz', 'w')
        np.savez(f, activations)
        f.close()